# Código Gerador de template do Arquivo 9805

#### 0000 - Import das Bibliotecas

In [18]:
import pandas as pd
import numpy as np

##### 0002B - Função Contador

In [19]:
def counter (x):
    if x>=0.0:
        x=1
    else:
        x=0
    return x

#### 0001 - Carregamento do Arquivo Excel

In [20]:
base = pd.read_csv('Pasta2.csv', sep=';', decimal=',')
base.cpf = base.cpf.apply(lambda x: counter(x) )
base.cnpj = base.cnpj.apply(lambda x: counter(x) )

In [26]:
try:
    base.drop(columns=['infAdicionais','id'], inplace=True)
except:
    pass

base = base.groupby(['tipoEnvio','dataBase','cnpjIf','codOrigem','modalidadePagamento']).sum()
base.reset_index(inplace=True)

#### 0002 - Função de Verificação se 'Not a Number'

In [23]:
def is_nan(value):
    return value!=value


#### 0003 - Confecção do Arquivo

In [24]:
for x in base.cnpjIf.unique():
    for y in base.dataBase.unique():
        for z in base.tipoEnvio.unique():
            if len(base[(base.cnpjIf == x)&(base.dataBase == y)&(base.tipoEnvio == z)]) >0:
                data = base[(base.cnpjIf == x)&(base.dataBase == y)&(base.tipoEnvio == z)].copy()
                cnpjif_string = str(int(x))
                while len(cnpjif_string)<8:
                    cnpjif_string = '0' + cnpjif_string
                file_name ='9805_'+cnpjif_string +'_'+ str(z) +'_'+str(y) +'.xml'
                arquivo = open(file_name, 'w+',encoding='utf8')

                #Preenchimento do cabeçalho
                arquivo.write('<?xml version="1.0" encoding="UTF-8"?>\n')
                arquivo.write('<asvr9805>\n')
                arquivo.write('   <codDoc>9805</codDoc>\n')
                arquivo.write('   <tipoEnvio>'+str(z)+'</tipoEnvio>\n')
                arquivo.write('   <dataBase>'+str(y)+'</dataBase>\n')
                
                arquivo.write('   <cnpjIf>'+cnpjif_string+'</cnpjIf>\n')


                #Preenchimento dos dados
                for w in data.index:
                    arquivo.write('    <valor>\n')

                    #preenchimento dados
                    valor_convertido_float = float(data.valorADevolver[w])
                    valor_convertido_string = "{:.2f}".format(valor_convertido_float)
                    arquivo.write('        <origem>'+'0' + str(data.codOrigem[w])+'</origem>\n')
                    arquivo.write('        <modalidadePagamento>'+ str(data.modalidadePagamento[w])+'</modalidadePagamento>\n')
                    arquivo.write('        <valorDevolvido>'+valor_convertido_string+'</valorDevolvido>\n')

                    if is_nan(data.cnpj[w]):
                        cpf_int = int(data.cpf[w])
                        cpf_string = str(cpf_int)
                        while len(cpf_string)<11:
                            cpf_string = '0' + cpf_string
                        arquivo.write('        <qtdCpfsBeneficiados>'+cpf_string+'</qtdCpfsBeneficiados>\n')
                    else:
                        cnpj_int = int(data.cnpj[w])
                        cnpj_string = str(cnpj_int)
                        while len(cnpj_string)<14:
                            cnpj_string = '0' + cnpj_string
                        arquivo.write('        <qtdCnpjsBeneficiados>'+str(cnpj_string)+'</qtdCnpjsBeneficiados>\n')


                    arquivo.write('    </valor>\n')
                arquivo.write('</asvr9805>\n')

                arquivo.close()

AttributeError: 'DataFrame' object has no attribute 'id'